In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_DEFAULT_MODEL"] = "gpt-4o-mini"

# Text Generation

In [ ]:
from openai import OpenAI

client = OpenAI()

respnose = client.responses.create(
    model="gpt-4o-mini",
    input="Write a haiku about the purple."
)

print(respnose.output_text)

In [ ]:
print(respnose.model_dump_json(indent=2))

In [ ]:
# instructions - developer, user, assistant

from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    instructions="Talk like a pirate.",
    input="Are indentation is compulsory in python?",
)

print(response.output_text)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content":"Talk like a pirate."
        },
        {
            "role": "user",
            "content": "What is it mean that talk like a pirate?"
        }
    ]
)

print(response.output_text)

# Images and Vision

In [ ]:
# generating image

import base64

respnose = client.responses.create(
    model = "gpt-4o-mini",
    input="Generate image of the Lord Krishna.",
    tools=[{"type": "image_generation"}], 
)

image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_base64 = image_data[0]
    with open("krishna.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

In [ ]:
# analyze the image - by passing a url

response = client.responses.create(
    model="gpt-4o-mini",
    input=[{
        "role": "user",
        "content":[
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
            }
        ]
    }]
)


print(response.output_text)

In [ ]:
# analyze the image - by passing a base64 encoded image

import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "panther.jpeg"

base64_image = encode_image(image_path)

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "what's in this image?"},
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}"
                }
            ]
        }
    ]
)

print(response.output_text)

In [ ]:
# # analyze the image - by passing a file id

import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "panther.jpeg"

base64_image = encode_image(image_path)

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "what's in this image?"},
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}"
                }
            ]
        }
    ]
)

print(respnose.output_text)

def create_file(file_path):
    with open(file_path, "rb") as file_content:
        result = client.files.create(
            file=file_content,
            purpose="vision", 
        )
        return result.id 
    
file_id = create_file("panther.jpeg")

response = client.responses.create( 
    model="gpt-4o-mini", 
    input=[{
        "role": "user", 
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "file_id": file_id
            }
        ]
    }]
)

print(response.output_text)

# Audio and Speech

In [ ]:
# audio output from model

import base64

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format":"wav"},
    messages=[
        {
            "role": "user",
            "content": "Tell me about rainbow."
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("rainbow.wav", "wb") as f:
    f.write(wav_bytes)

In [ ]:
!pip install -q requests

In [ ]:
# audio input to model

import requests

url = "https://cdn.openai.com/API/docs/audio/alloy.wav"
respnose = requests.get(url)
respnose.raise_for_status()
wav_data = respnose.content
encoded_string = base64.b64encode(wav_data).decode('utf-8')

completion = client.chat.completions.create(
    model = "gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "what is in this recording?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "wav"
                    }
                }
            ]
        }
    ]
)

print(completion.choices[0].message)


# Structured Output

In [ ]:
from pydantic import BaseModel

class CalenderEvent(BaseModel):
    name: str
    date: str 
    participants: list[str]

respnose = client.responses.parse(
    model = "gpt-4o-mini",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday"
        }
    ],
    text_format=CalenderEvent
)

event = respnose.output_parsed

print(event)

In [ ]:
# chain of thought

class Step(BaseModel):
    explanation: str 
    output: str 

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str 

response = client.responses.parse(
    model = "gpt-4o-mini",
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step."
        },
        {
            "role": "user",
            "content": "how can I solve 8x + 7 = -23"
        }
    ],
    text_format=MathReasoning
)

math_reasonsing = response.output_parsed

print(math_reasonsing)

In [ ]:
# structured data extraction

class ReasearchPaperExtraction(BaseModel):
    title: str 
    authors: list[str]
    abstract: str 
    keywords: list[str]

response = client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {
            "role": "system",
            "content": "You are a expert at structured data extraction. You will be given unstructured text from a reasearch paper and should convert it into the given structre."
        },
        {
            "role": "user",
            "content": ""
        }
    ],
    text_format=ReasearchPaperExtraction
)

research_paper = response.output_parsed

In [ ]:
# ui generation

from enum import Enum 
from typing import List 
